<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-requests,-BeautifulSoup,-pandas-and-time" data-toc-modified-id="Import-requests,-BeautifulSoup,-pandas-and-time-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import requests, BeautifulSoup, pandas and time</a></span></li><li><span><a href="#Web-scraping" data-toc-modified-id="Web-scraping-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Web scraping</a></span><ul class="toc-item"><li><span><a href="#Creat-a-list-from-a-to-w-and-y" data-toc-modified-id="Creat-a-list-from-a-to-w-and-y-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Creat a list from a to w and y</a></span></li><li><span><a href="#Scrape-the-name,-URL,-address-and-contact-information-of-daycare-centers-of-Toronto" data-toc-modified-id="Scrape-the-name,-URL,-address-and-contact-information-of-daycare-centers-of-Toronto-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Scrape the name, URL, address and contact information of daycare centers of Toronto</a></span></li></ul></li><li><span><a href="#Split-the-Add-and-Contact-Information-columns-into-four-columns" data-toc-modified-id="Split-the-Add-and-Contact-Information-columns-into-four-columns-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Split the Add and Contact Information columns into four columns</a></span></li><li><span><a href="#Save-the-dataframe-to-an-csv-file" data-toc-modified-id="Save-the-dataframe-to-an-csv-file-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Save the dataframe to an csv file</a></span></li></ul></div>

A common way to gather data off the web is to scrape it!  Let’s try our hand at some!  As you have tried in other parts of the course we will use beautifulsoup to grab some daycare data off the toronto.ca website.

https://www.toronto.ca/data/children/dmc/a2z/a2za.html - a comprehensive list of licensed day cares in Toronto.

You have decided to use some of your new found web scraping and data analysis skills to help narrow down your search for daycare services in Toronto.

Use the python packages “requests” and “BeautifulSoup” in order to scrape the above URL to get the daycare name and address.  Save the results into a pandas dataframe and save it as results.csv for later use.

Your pandas data frame should consist of the following info:
1.	Name of Daycare
2.	Address of Daycare
3.	Daycare centre contact information

Please be sure to scrape all letters of the alphabet!

In order to be considerate to the hosts of www.toronto.ca please incorporate a sleep of (5 seconds) in between requests!
Hand in:
1.	Copy of your pandas dataframe csv
2.	Copy of your python file


# Import requests, BeautifulSoup, pandas and time

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import re

# Web scraping

## Creat a list from a to w and y

In [8]:
atoy = [chr(i) for i in range(ord('a'),ord('w')+1)]
atoy.append('y')
print(atoy)


['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'y']


## Scrape the name, URL, address and contact information of daycare centers of Toronto

In [9]:
def daycarelist(url):

    #---Lists to store the scraped data in---
    dcnames = []
    dcadds = []
    dcinfos = []
    #---Sleep for 5 seconds---
    time.sleep(5)
    #---Get the web source code ---
    r = requests.get(url)
    source = r.content
    soup = BeautifulSoup(source,"html.parser")
    soup_new = soup.find('div',id='pfrBody')

    for daycare in soup_new.find_all('a'):
        if 'webreg'in daycare.attrs['href']:
            dcname = daycare.text
            dcnames.append(dcname)
            dcurl = f"https://www.toronto.ca/data/children/dmc{daycare.attrs['href'].replace('..','')}"
            #---Sleep for 5 seconds---
            time.sleep(5)
            #--- Scraping the address of a daycare center ---
            r2 = requests.get(dcurl)
            source2 = r2.content
            soup2 = BeautifulSoup(source2,"html.parser")
            soup_new2 = soup2.find('div',attrs={'class':"csd_opcrit_content_box"})
            dcad = soup_new2.find('p').text.replace(' ',' ')
            #--- Delete space---
            dcad = re.sub(r'\s',' ',dcad)
            dcadds.append(dcad)    
            #---Sleep for 5 seconds---
            time.sleep(5)
            #--- Scraping the contact information of a daycare center ---
            r3 = requests.get(dcurl)
            source3 = r3.content
            soup3 = BeautifulSoup(source3,"html.parser")
            dcinfo = soup3.find('li',attrs={'class':"nudge"})
            #--- Some daycare centers have no contact information ---
            if dcinfo == None:
                dcinfo = 'No contact information.'
                dcinfos.append(dcinfo)
            else:
                dcinfo =dcinfo.text.replace("Phone:","")
                #--- Delete enter and space---
                dcinfo = dcinfo.strip(" ,\r,\t\,\n")
                dcinfo = re.sub(r'\s',' ',dcinfo)
                dcinfo = dcinfo.replace(',                    ',': ')
                dcinfo = dcinfo.replace('                       ',' ')            
                dcinfos.append(dcinfo)  
            # ---Put the string into a dataframe---
            df=pd.DataFrame()
            df['Daycare Centre Name'] = dcnames
            df['Add'] = dcadds
            df['Contact Information'] = dcinfos
    return df

#---Create a dataframe of 3 columns---
dc=pd.DataFrame({},columns=['Daycare Centre Name','Add','Contact Information'])
#---Loop a to w and y lists---
for list in atoy:
    url = 'https://www.toronto.ca/data/children/dmc/a2z/a2z'+list+'.html'
    tmp = daycarelist(url)
    dc = pd.concat([dc,tmp])
dc


,Daycare Centre Name,Add,Contact Information
0,A & A Child Care Centre,"3380 Midland Ave ,22 (Midland / Finch E.) ...",Angela McAloney: 416-298-4489
1,Abacus Montessori Learning Centre,2723 St Clair Ave E Ward: Beaches-East York,Jayanthi Rajagopalan: 416-285-0110
2,ABC Academy - 18 Melrose Avenue,18 Melrose Ave (Yonge / Lawrence E.) War...,Jennifer Halliwell: 416-485-5437
3,ABC Academy 2 - 20 Melrose Avenue,20 Melrose Ave (Yonge / Lawrence E.) War...,Jennifer Halliwell: 416-485-5437
4,ABC Academy Bayview - 1432 Bayview Avenue,1432 Bayview Ave (Bayview Ave/Davisville Ave...,Helen Brydges: 647-349-2229
...,...,...,...
7,York Mills Child Care Centre,1200 York Mills Rd (York Mills / Dvp) Wa...,Najma Jabeen: 416-449-9453
8,Yorktown Montessori School,349 Kenneth Ave (Yonge / Finch) Ward: Wi...,Grace Park: 416-226-6932
9,York University Co-Operative Day Care Centre,"90 Atkinson Rd ,Rm 128 (Finch / Keele) W...",Mabel Davis: 416-736-5190 ext 33273
10,Yorkview Kids Care,130 Yorkview Dr (Bathurst / Finch) Ward:...,Ruth Barber: 416-221-7656


# Split the Add and Contact Information columns into four columns

In [10]:
dc[["Address","Ward"]]=dc["Add"].str.split("Ward:",expand = True)
dc[["Contact Name","Contact Phone"]]=dc["Contact Information"].str.split(":",expand = True)
dc.drop(columns=["Add","Contact Information"],inplace = True)
dc

,Daycare Centre Name,Address,Ward,Contact Name,Contact Phone
0,A & A Child Care Centre,"3380 Midland Ave ,22 (Midland / Finch E.)",Scarborough-Agincourt,Angela McAloney,416-298-4489
1,Abacus Montessori Learning Centre,2723 St Clair Ave E,Beaches-East York,Jayanthi Rajagopalan,416-285-0110
2,ABC Academy - 18 Melrose Avenue,18 Melrose Ave (Yonge / Lawrence E.),Eglinton-Lawrence,Jennifer Halliwell,416-485-5437
3,ABC Academy 2 - 20 Melrose Avenue,20 Melrose Ave (Yonge / Lawrence E.),Eglinton-Lawrence,Jennifer Halliwell,416-485-5437
4,ABC Academy Bayview - 1432 Bayview Avenue,1432 Bayview Ave (Bayview Ave/Davisville Ave...,Don Valley West,Helen Brydges,647-349-2229
...,...,...,...,...,...
7,York Mills Child Care Centre,1200 York Mills Rd (York Mills / Dvp),Don Valley East,Najma Jabeen,416-449-9453
8,Yorktown Montessori School,349 Kenneth Ave (Yonge / Finch),Willowdale,Grace Park,416-226-6932
9,York University Co-Operative Day Care Centre,"90 Atkinson Rd ,Rm 128 (Finch / Keele)",Humber River-Black Creek,Mabel Davis,416-736-5190 ext 33273
10,Yorkview Kids Care,130 Yorkview Dr (Bathurst / Finch),Willowdale,Ruth Barber,416-221-7656


# Save the dataframe to an csv file

In [11]:
#---Save the dataframe to an csv file---
dc.to_csv('results.csv',index=False)